# Исследование объявлений о продаже квартир

В распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Задача проекта — установить параметры, так как это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность. 

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. 

### Общая информация.

In [1]:
import pandas as pd
#подключаем библиотеки для построения графиков
import matplotlib.pyplot as plt
import seaborn as sns
data = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')
pd.set_option('display.max_columns', None)
data

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/real_estate_data.csv'

In [ ]:
data.hist(figsize = (15,20));

Вызывают подозрения гистограммы по столбцам: Высота потолков, цена объекта, расстояние до центра города, расстояние до парка около 3000 м, расстояние до водоема около 3000м.

### Предобработка данных

Определение в каких стобцах есть пропуски можно методом isna().sum() или методом value_counts(). В каждом столбце. Также можно еще проверить сколько вообще строк в таблице методом shape() потом вызвать методом info() посмотреть в каких столбцах есть пропуски чтобы заранее знать.

In [ ]:
data.info() 
data.shape

In [ ]:
data.isnull().sum() #изучаем количество пропусков

Пропуски есть в столбцах: ceiling_height,floors_total ,living_area, is_apartment,kitchen_area,balcony, airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest ,days_exposition. ceiling_height - неизвестно как сейчас обработать floors_total - неизвестно как сейчас обработать living_area - Обработать среднее значение.

***Столбец Ceiling_height***

С помощью цикла уберем выделяющиеся значения делением на 10.

In [ ]:
for i in data['ceiling_height']:
        if i > 10 and i in data['ceiling_height']:
            i = i // 10
data['ceiling_height'].sort_values(ascending=False)

In [ ]:
data['ceiling_height'] = data['ceiling_height'].fillna(data['ceiling_height'].median()) #Замена пустых значений на медианное значение высоты потолка
data['ceiling_height'].isnull().sum() #проверка остались ли пустые значения

***Столбец floors_total***

Здесь нельзя вставить ни медианное, ни среднее значение, так как количество этажей во всем доме либо есть, либо его нет. Тогда где пропуски, нужно удалить эти строки (эти пустые значения) методом dropna()

In [ ]:
data = data.dropna(subset = ['floors_total']) #удаляем значения NaN

In [ ]:
data['floors_total'].isnull().sum() #проверка, что удалились все пропуски

***Столбец is_apartment.*** Подразумевается, что человек, когда отвечал на вопрос: "Это апартаменты?", пропуском он подразумевал False

In [ ]:
data['is_apartment'] = data['is_apartment'].fillna(False)
data['is_apartment'].isnull().sum() #проверка, что заменили все пропуски

***Столбец living_area***

Жилая площадь не должна быть меньше 6.00 по гос.стандарту.

In [ ]:
data.query('living_area > 6') # находим срез удовлятворяющей  условию гос.стандарта (минимальная жилая плошадь 6.00)

In [ ]:
data['living_area'].sort_values() #проверка, что убрались значения, неудовлетворяющие нашему условию

***Столбец kitchen_area***

Площадь кухни не может быть меньше 4.50 метров по гос.стандарту

In [ ]:
data = data.query('kitchen_area >4.50') # находим срез удовлятворяющий условию гос.стандарта

In [ ]:
data['kitchen_area'].sort_values() # проверяем что нет значений, которые меньшe 4.5

***Столбец balcony*** Нужно заполнить пустые значения медианным значением площади балкона.

In [ ]:
data['balcony'] = data['balcony'].fillna(0)

In [ ]:
data['balcony'].head()

In [ ]:
data.info() #проверка что заменили все значения и не удалили ничего лишнего.

Удалилось с изначального датафрейма где-то 2831 строчек, которые не подходили по каким-то критериям

In [ ]:
data.isnull().sum()

***Столбец locality_name***

Сначала удалим пропуски, где не написано название поселка.

In [ ]:
data = data.dropna(subset = ['locality_name'])
data['locality_name'].isnull().sum() #проверка, что удалили пустые значения

In [ ]:
data['locality_name'].unique() #посмотрим все варианты уникальных значений

In [ ]:
#функция распределяющая все названия на поселки, села, деревни, пгт, садовые товарищества и прочие(тут города и модет названия поселков где не указано типа)
def type_locality_name(locality):
    if 'посёлок' in locality:
        return 'поселок'
    if 'поселок' in locality:
        return 'поселок'
    if 'поселок при железнодорожной станции' in locality or 'посёлок при железнодорожной станции' in locality:
        return 'поселок'
    if 'деревня' in locality:
        return 'деревня'
    if 'село' in locality:
        return 'село'
    if 'посёлок городского типа'  in locality or 'поселок городского типа' in locality:
        return 'пгт'
    if 'городской посёлок' in locality or 'городской поселок' in locality:
        return 'пгт'
    if 'садовое товарищество' in locality or 'садовое некоммерческое товарищество' in locality:
        return 'садовое товарищество'
    else:
        return 'прочие'

In [ ]:
data['locality_type'] = data['locality_name'].apply(type_locality_name)

In [ ]:
data.head()

In [ ]:
# Поменяем в столбце с названиями типы поселков и е на ё сначала
# Нам нужно убрать название поселка, города и т.д. 
data['locality_name'] = data['locality_name'].replace('|'.join(['посёлок ', 'поселок ','городского ','типа ','деревня ']), '', regex=True)

In [ ]:
data['locality_name'].value_counts()

Заполним пропуски в картографических данных "-1", что будет означать что здесь пропуск.

***Столбец airports_nearest***

In [ ]:
data['airports_nearest'] = data['airports_nearest'].fillna(-1)

***Столбец cityCenters_nearest***

In [ ]:
data['cityCenters_nearest'] = data['cityCenters_nearest'].fillna(-1)

***Столбец parks_around3000***

In [ ]:
data['parks_around3000'] = data['parks_around3000'].fillna(-1)

***Столбец parks_nearest***

In [ ]:
data['parks_nearest'] = data['parks_nearest'].fillna(-1)

***Столбец ponds_around3000***

In [ ]:
data['ponds_around3000'] = data['ponds_around3000'].fillna(-1)

***Столбец ponds_nearest***

In [ ]:
data['ponds_nearest'] = data['ponds_nearest'].fillna(-1)

Тоже заполним столбец "сколько дней было размещено объявление (от публикации до снятия)" -1, что покажет что квартира еще не продана.

***Столбец days_exposition***

In [ ]:
data['days_exposition'] = data['days_exposition'].fillna(-1)

In [ ]:
data

Поменяем типы данных. Столбец first_day_exposition с строки в тип дат. Столбец floors_total с вещественных в целый. Столбец balcony с вещественных в целый.

In [ ]:
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format = '%Y-%m-%dT%H:%M:%S')
data['floors_total'] = data['floors_total'].astype('int')
data['balcony'] = data['balcony'].astype('int')

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
#код ревьюера
data['total_area'].hist(bins=30);

***Total_area***

Считаем аномальными квартиры, которые больше 250 кв.м

In [ ]:
data = data.query('total_area <= 250')
data['total_area'].sort_values(ascending=False) #проверка, что нет значений больше 250

***Ceiling_height***

In [ ]:
data['ceiling_height'].hist(bins=15);


Считаем аномальными квартиры, у которых потолок выше 6м

In [ ]:
data = data.query('ceiling_height <= 6')
data['total_area'].sort_values(ascending=False) #проверка, что нет значений больше 6

***Rooms***

In [ ]:
data['rooms'].hist(bins=30);

Аномальными значениями можно считать квартиры, где комнат больше 7.

In [ ]:
data = data.query('rooms <= 7')
data['rooms'].sort_values(ascending=False) #проверка, что нет значений больше 7

Завершилась предработка данных. Из исходной таблицы удалила 2831 строку. Они не подходили по описанию и по заданной мной параметрам. Также обрабботала все неявные дубликаты и заполнила пропуски, соответсвующими значениями. 

### Добавьте в таблицу новые столбцы со следующими параметрами

Добавление столбца: цена одного квадратного метра
Цена 1 м кв= last_price/total_area

In [ ]:
data['metres_price'] = data['last_price'] / data['total_area']

In [ ]:
data['metres_price'].head()

Добавление столбца: день недели публикации объявления (0 — понедельник, 1 — вторник и так далее). Через функцию. Метод dayofweek возвращает номер дня недели.

In [ ]:
def day_of_week(days):
    if days.dayofweek == 0:
        return "понедельник" 
    elif days.dayofweek == 1:
        return "вторник"
    elif days.dayofweek == 2:
        return "среда"
    elif days.dayofweek == 3:
        return "четверг"
    elif days.dayofweek == 4:
        return "пятница"
    elif days.dayofweek == 5:
        return "суббота"
    elif days.dayofweek == 6:
        return "воскресенье"
data['day_of_week'] = data['first_day_exposition'].apply(day_of_week)

In [ ]:
data.head(25)

Добавление столбца: месяц публикации объявления. По аналогии с днем недели

In [ ]:
def name_month(months):
    if months.month == 1:
        return "январь" 
    elif months.month == 2:
        return "февраль" 
    elif months.month == 3:
        return "март"
    elif months.month == 4:
        return "апрель"
    elif months.month == 5:
        return "май"
    elif months.month == 6:
        return "июнь"
    elif months.month == 7:
        return "июль"
    elif months.month == 8:
        return "август"
    elif months.month == 9:
        return "сентябрь"
    elif months.month == 10:
        return "октябрь"
    elif months.month == 11:
        return "ноябрь"
    elif months.month == 12:
        return "декабрь"
data['months_name'] = data['first_day_exposition'].apply(name_month)

Добавление столбца: год публикации объявления

In [ ]:
data['years'] = data['first_day_exposition'].dt.year

In [ ]:
data

Добавление столбца: тип этажа квартиры (значения — «первый», «последний», «другой»). С помощью функции

In [ ]:
data['floors_total'].sort_values(ascending=False) 

Максимальный этаж 60

In [ ]:
data['floor'].unique()

In [ ]:
data['floors_total'].unique()

In [ ]:
def type_of_floor(floor, total):
    if floor == 1:
        return "первый"
    elif floor == total:
        return "последний"
    else:
        return "другой"

In [ ]:
data['type_of_floor'] = data.apply(lambda x: type_of_floor(x['floor'], x['floors_total']), axis=1)

Добавление столбца: расстояние до центра города в километрах

In [ ]:
data['cityCenters_nearest_km'] = (data['cityCenters_nearest'] / 1000).round()

In [ ]:
data.head(50)

### Проведите исследовательский анализ данных

Найдем взаимость с каждым из данных параметров.

Сначала рассмотрим каждый параметр по отдельности. Построим для него гистограмму. Применим метод describe(), чтобы узнать минимум, максимум, "усы", выбросы. Построим гистограмму, диаграмму размаха. 


#### Рассмотрим сначала каждый параметр по отдельности. А потом их зависимости.

Общая площадь объекта

In [ ]:
data['total_area'].describe()

In [ ]:
data.boxplot('total_area');

In [ ]:
data['total_area'].hist(bins=100); #строим гистограмму

Большее количество квартир собрано в начале, т. есть у них площадь примерно медианное значение. И таких кватртир большинство

Жилая площадь

In [ ]:
data['living_area'].describe();

In [ ]:
data.boxplot('living_area');

In [ ]:
data['living_area'].hist(bins=30);

Анологично общей площади, большинство квартир име/т медианное значение жилой площади, так же уществуют и выбросы.

Площадь кухни

In [ ]:
data['kitchen_area'].describe();

In [ ]:
data.boxplot('kitchen_area');

In [ ]:
data['kitchen_area'].hist(bins=40);

Большенство кватрир имеют среднюю площадь кухни, есть и те, которые выбиваются, но их значительно меньше

Цена объекта

In [ ]:
data['last_price'] = data['last_price'] / 1000000

In [ ]:
data['last_price'].describe()

In [ ]:
data.boxplot('last_price');

In [ ]:
data['last_price'].hist(bins=100);

Цены на квартиры абсолютно разные, но чаще встречаемая цена это около 5 млн. Че выше цена, тем меньше представленных объектов

Количество комнат

In [ ]:
data['rooms'].describe()

In [ ]:
data.boxplot('rooms');

In [ ]:
data['rooms'].hist(bins=50);

Почти одинаковое количество наиболее встречаемях однокомнатных и двухклмнатных квартир. Далше идут трешки, и чем больше комнат, тем меньше представвленных квартир.

Высота потолков

In [ ]:
data['ceiling_height'].describe()

In [ ]:
data.boxplot('ceiling_height');

In [ ]:
data['ceiling_height'].hist(bins=20);

В основном в квартирах высота потолка где-то 3 м. Все, что выше 7 м, выбросы

In [ ]:
data['floor'].describe()

In [ ]:
data.boxplot('floor');

In [ ]:
data['floor'].hist(bins=40);

Большинство квартир на 2-3 этаже. Чем выше этаж, тем меньше представленных квартир

Тип этажа квартиры («первый», «последний», «другой»)

In [ ]:
data['type_of_floor'].describe()

In [ ]:
data['type_of_floor'].hist(bins=40);

Большее количество квартир находятся на средних этажах.

Общее количество этажей в доме

In [ ]:
data['floors_total'].describe()

In [ ]:
data.boxplot('floors_total');

In [ ]:
data['floors_total'].hist(bins=40);

В домах, мног квартир на пятом этаже, потом на 9. Чем больше этаже в доме, тем меньше квартир.

Расстояние до центра города в метрах

In [ ]:
data['cityCenters_nearest'].describe();

In [ ]:
data.boxplot('cityCenters_nearest');

In [ ]:
data['cityCenters_nearest'].hist(bins=40);

Есть квартиры,которые находятся в центре, их большинство. Также находятся объекты  в 10-20 км от центра

Расстояние до ближайшего аэропорта

In [ ]:
data['airports_nearest'].describe()

In [ ]:
data.boxplot('airports_nearest');

In [ ]:
data['airports_nearest'].hist(bins=40);

Помним, что 0 мы обозначали пропуски. Среднее расстояние до аэропорта где то 20 км

Расстояние до ближайшего парка

In [ ]:
data['parks_nearest'].describe()

In [ ]:
data.boxplot('parks_nearest');

In [ ]:
data['parks_nearest'].hist(bins=40);

В среднем объекты расположены в 500 м от парка. И чем больше расстояние от парка, тем менше представляемых квартир

День публикации объявления

In [ ]:
data['day_of_week'].describe()

In [ ]:
data['day_of_week'].hist(bins=7, figsize=(12,10));

В четверг выставили большее количество квартир. А в вс меньшее. В выходные мало кто выставляет на продажу, чем в простые дни.

месяц публикации объявления

In [ ]:
data['months_name'].describe()

In [ ]:
data['months_name'].hist(bins=12, figsize=(12,10));


В марте было высталвено больше всего объявлений. А в декабре и мае наименьшее количество

Теперь изучим, как быстро продавались квартиры

In [ ]:
data[data['days_exposition'] == -1]['days_exposition'].count() 

In [ ]:
data[data['days_exposition'] != -1]['days_exposition'].mean()

In [ ]:
data[data['days_exposition'] != -1]['days_exposition'].median()

In [ ]:
data[data['days_exposition'] != -1]['days_exposition'].max()

In [ ]:
data[data['days_exposition'] != -1]['days_exposition'].min()

In [ ]:
data[data['days_exposition'] != -1]['days_exposition'].hist(bins=50);

In [ ]:
data[data['days_exposition'] != -1]['days_exposition'].hist(bins=50, range=(1, 102));

In [ ]:
data[data['days_exposition'] != -1]['days_exposition'].hist(bins=50, range=(102, 1580));

В среднем объявление находится на сайте около 110-150 дней. До медианного значения есть два пика, это 45-50 дней и 60 дней. В общем объявления убывают по количеству до 100 дней, потом самы пик, самое большое количество объявлений, а потом они уже резко убывают. Самыми быстрыми можно считать до 20 дней, самыми долгими после 1200 дней, так как их очень мало.

Изучим зависимости цены от данных параметров

In [ ]:
data.pivot_table(index = 'total_area', values='last_price').plot();

In [ ]:
data.pivot_table(index = 'total_area', values='last_price').hist(bins=6);

In [ ]:
data.plot(x = 'total_area', y='last_price', kind='scatter', grid=True);

Чем больше площадь, тем больше цена.

Жилая площадь от цены

In [ ]:
data.pivot_table(index = 'living_area', values='last_price').plot();

In [ ]:
data.pivot_table(index = 'living_area', values='last_price').hist(bins=20);

In [ ]:
data.plot(x = 'living_area', y='last_price', kind='scatter', grid=True);

На графиках видно, что нет прямой зависимоси жилой площади от цены

Площадь кухни и цена

In [ ]:
data.pivot_table(index = 'kitchen_area', values='last_price').plot();

In [ ]:
data.pivot_table(index = 'kitchen_area', values='last_price').hist(bins=20);

In [ ]:
data.plot(x = 'kitchen_area', y='last_price', kind='scatter', grid=True);

Не видно прямой зависимости от стоимости и пощади кухни

Количество комнат и цена

In [ ]:
data.pivot_table(index = 'rooms', values='last_price').plot();

In [ ]:
data.pivot_table(index = 'rooms', values='last_price').hist(bins=20);

In [ ]:
data.plot(x = 'rooms', y='last_price', kind='scatter', grid=True);

Нет явной зависимости от числа комнат и цены

In [ ]:
data[['total_area', 'last_price', 'living_area', 'kitchen_area', 'rooms']].corr()

In [ ]:
data.pivot_table(index = 'type_of_floor', values='last_price').plot();

In [ ]:
data.pivot_table(index = 'type_of_floor', values='last_price').plot.bar();

Чем выше этаж, тем больше стоимость

День недели и цена

In [ ]:
data.pivot_table(index = 'day_of_week', values='last_price').plot();

In [ ]:
data.pivot_table(index = 'day_of_week', values='last_price').plot(kind='bar',figsize=(12,10));

В пятницу были самые худшие продажи

Месяц и цена

In [ ]:
data.pivot_table(index = 'months_name',values='last_price').plot(figsize=(12,10), grid=True);

In [ ]:
data.pivot_table(index = 'months_name',values='last_price').plot(kind='bar', figsize=(12,10), grid=True);

Хуже продаются квартиры в октябре

Год и цена

In [ ]:
data.pivot_table(index = 'years',values='last_price').plot(kind='bar', figsize=(12,10), grid=True);

In [ ]:
data.pivot_table(index = 'years',values='last_price').plot( figsize=(12,10), grid=True);

Наибольшее количество продаж было в 2014 году, а дальше все начало снижаться

Найдем наиболее чаще встречаемых населенных пунктов, и найдем там среднюю цену кв м в каждом таком населенном пункте

In [ ]:
data['locality_name'].value_counts().head(10) 

Нашли топ 10 частовстречаемых объявлений

In [ ]:
#создадим отдельный датафрейм в котором найдем эти города
sr = data.query('(locality_name == "Санкт-Петербург") or (locality_name == "Мурино")  or (locality_name == "Шушары") or (locality_name == "Кудрово" ) or (locality_name == "Всеволожск" ) or (locality_name == "Пушкин") or (locality_name == "Колпино") or (locality_name == "Гатчина") or (locality_name == "Парголово") or (locality_name == "Петергоф")') #############)
res = sr.pivot_table(index='locality_name', values='metres_price', aggfunc='mean')
res

In [ ]:
print(res.min())
res.max()

In [ ]:
res.value_counts()

Топ 1 - Санкт-Петербург
Топ10 - Гатчина
Минимальная цена из 10 чатовстречаемых городов в Гатчине, максимальная в Санкт_петербурге

Санкт-Петербург

In [ ]:
saint = data.query('locality_name == "Санкт-Петербург"')
print(saint.pivot_table(index='cityCenters_nearest_km',values='last_price', aggfunc='mean'))
saint.pivot_table(index='cityCenters_nearest_km',values='last_price', aggfunc='mean').plot();

Видно, что чем дальше от центра находится квартира, тем она дешевле.

Центр города приблизително заканчивается на 5 км

Центр города приблизително заканчивается после 6-7км

### Общий вывод

# Вывод

Я сделала предработку данных и нашла интересные зависимости на рынке:
- В предработке данных я уменьшила изначальную таблицу где-то на 10%. 
- Убрала пропущенные значения (сколько этажей в здании ) и аномальные значения, не соответсвующие гос.стандартам. Такие как:площадь кухни меньше 4.50 метра; жилая площадь меньше 7.00; общая площадь больше 250 и т.д. Также заполнила картографические пропуски значением -1, что означает пропущенное значение. Поменяла тип данных в некоторых столбцах, чтобы в дальнейшем с ними было удобней раотать. Так завершилась предработка данных. 
- Дальше я добавила несколько столбцов, которые помогут в дальнейшем исследовании проверить некоторые зависимости. 
- Зависимость между общей площадью и ценой объекта достаточно явная. Чем больше площадь, тем больше цена объекта. 
- Также достаточно явная зависимость наблюдается между на каком этаже находится квартира и ценой объекта. Чем выше находится квартира, тем больше цена. 
- Еще некоторые интересные зависимости: 
    - Самый худший днем недели для начала продаж является пятница. Потом уже идут выходные дни. Как ни странно  самый худший месяц это октябрь, а год 2018. 
    - Самым благоприятным из предоставленных считается понедельник в декабре в 2014 году. 
    - Из 10 самых встречаемых объектов является Санкт-Петербург, также в этом городе самая большая цена за квадратный метр. 
    - Интересная зависимость, что чем дальше от центра города находится квартира, тем  цена меньше. 
    
В итоге я выявила интересные зависимости на рынке недвижимости.
